# Advanced NumPy

```{admonition} Information
:class: info

**Prerequisites:** Module 02 Lessons 1-2 (Python Fundamentals, NumPy Basics)  
**Learning Objectives:**
- Work with sparse matrices for large power system problems
- Apply advanced linear algebra techniques
- Optimize NumPy code for performance
- Prepare for numerical methods in Module 04
- Handle large-scale power system data efficiently

**Estimated Time:** 2 hours
```

## Introduction

Power system analysis often involves large networks with thousands of buses and branches. These systems result in sparse matrices where most elements are zero. This lesson introduces advanced NumPy techniques and sparse matrix operations essential for efficient power system computation. These skills directly prepare you for the numerical methods covered in Module 04.

We'll explore how to leverage sparsity, perform efficient linear algebra operations, and optimize code for large-scale problems while maintaining the clarity and correctness required for engineering applications.

In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse import linalg as sp_linalg
import time
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

## 1. Introduction to Sparse Matrices

In power systems, network matrices are naturally sparse because each bus is only connected to a few neighboring buses. Understanding and exploiting this sparsity is crucial for computational efficiency.

### Why Sparse Matrices Matter

In [ ]:
# Compare memory usage: dense vs sparse
# Simulate a power system network matrix
n_buses = 1000
avg_connections = 3  # Average connections per bus

# Create a sparse connection pattern
# Each bus connects to avg_connections other buses
connections = []
for i in range(n_buses):
    # Connect to nearby buses
    for j in range(1, avg_connections + 1):
        if i + j < n_buses:
            connections.append((i, i + j))
            connections.append((i + j, i))  # Symmetric

# Remove duplicates
connections = list(set(connections))
n_connections = len(connections)

print(f"Network Statistics:")
print(f"Number of buses: {n_buses}")
print(f"Number of connections: {n_connections}")
print(f"Sparsity: {n_connections / (n_buses * n_buses) * 100:.2f}%")

# Create dense matrix
dense_matrix = np.zeros((n_buses, n_buses))
for i, j in connections:
    dense_matrix[i, j] = np.random.uniform(0.01, 0.1)

# Create sparse matrix
row_ind = [c[0] for c in connections]
col_ind = [c[1] for c in connections]
data = np.random.uniform(0.01, 0.1, n_connections)
sparse_matrix = sp.csr_matrix((data, (row_ind, col_ind)), 
                             shape=(n_buses, n_buses))

# Compare memory usage
dense_memory = dense_matrix.nbytes / 1024 / 1024  # MB
sparse_memory = (sparse_matrix.data.nbytes + 
                sparse_matrix.indices.nbytes + 
                sparse_matrix.indptr.nbytes) / 1024 / 1024  # MB

print(f"\nMemory Usage:")
print(f"Dense matrix: {dense_memory:.2f} MB")
print(f"Sparse matrix: {sparse_memory:.2f} MB")
print(f"Memory savings: {(1 - sparse_memory/dense_memory)*100:.1f}%")

### Sparse Matrix Formats

In [ ]:
# Different sparse matrix formats for different purposes
# Small example for visualization
n = 5
# Y-bus like structure: diagonal dominant with few off-diagonal elements
row = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4]
col = [0, 1, 0, 1, 2, 1, 2, 3, 2, 3, 4, 3, 4]
data = [3, -1, -1, 4, -2, -2, 5, -1, -1, 4, -2, -2, 3]

# Create different formats
coo = sp.coo_matrix((data, (row, col)), shape=(n, n))
csr = coo.tocsr()  # Compressed Sparse Row
csc = coo.tocsc()  # Compressed Sparse Column
dok = coo.todok()  # Dictionary of Keys
lil = coo.tolil()  # List of Lists

print("Sparse Matrix Formats:")
print("\n1. COO (Coordinate) Format:")
print(f"   Data: {coo.data}")
print(f"   Row indices: {coo.row}")
print(f"   Col indices: {coo.col}")

print("\n2. CSR (Compressed Sparse Row) Format:")
print(f"   Data: {csr.data}")
print(f"   Indices: {csr.indices}")
print(f"   Indptr: {csr.indptr}")

print("\n3. DOK (Dictionary of Keys) Format:")
print(f"   Dictionary: {dict(dok)}")

print("\nDense representation:")
print(csr.toarray())

# Format comparison for different operations
print("\nBest format for different operations:")
print("- Matrix construction: COO or DOK")
print("- Matrix-vector multiplication: CSR or CSC")
print("- Row slicing: CSR")
print("- Column slicing: CSC")
print("- Element-wise operations: COO")

### Building Sparse Matrices Incrementally

In [ ]:
# Building Y-bus matrix incrementally
# Typical pattern in power system analysis

def build_ybus_sparse(bus_data, branch_data):
    """
    Build Y-bus matrix using sparse format.
    bus_data: list of bus indices
    branch_data: list of (from_bus, to_bus, admittance)
    """
    n = len(bus_data)
    
    # Use DOK format for incremental construction
    ybus = sp.dok_matrix((n, n), dtype=complex)
    
    # Add branch admittances
    for from_bus, to_bus, y in branch_data:
        # Off-diagonal elements
        ybus[from_bus, to_bus] -= y
        ybus[to_bus, from_bus] -= y
        # Diagonal elements
        ybus[from_bus, from_bus] += y
        ybus[to_bus, to_bus] += y
    
    # Convert to CSR for efficient operations
    return ybus.tocsr()

# Example: 6-bus system
buses = list(range(6))
branches = [
    (0, 1, 2.0 - 6.0j),
    (0, 4, 1.0 - 3.0j),
    (1, 2, 1.5 - 4.5j),
    (1, 3, 1.0 - 3.0j),
    (1, 4, 0.5 - 1.5j),
    (2, 3, 2.0 - 6.0j),
    (3, 5, 1.0 - 3.0j),
    (4, 5, 1.5 - 4.5j)
]

ybus_sparse = build_ybus_sparse(buses, branches)

print("Y-bus matrix (sparse):")
print(f"Shape: {ybus_sparse.shape}")
print(f"Non-zero elements: {ybus_sparse.nnz}")
print(f"Sparsity: {ybus_sparse.nnz / (ybus_sparse.shape[0]**2) * 100:.1f}%")

# Visualize sparsity pattern
plt.figure(figsize=(6, 6))
plt.spy(ybus_sparse, markersize=10)
plt.title('Y-bus Sparsity Pattern')
plt.xlabel('Column Index')
plt.ylabel('Row Index')
plt.show()

# Show a few elements
print("\nSample elements:")
print(f"Y[0,0] = {ybus_sparse[0,0]:.3f}")
print(f"Y[0,1] = {ybus_sparse[0,1]:.3f}")
print(f"Y[1,1] = {ybus_sparse[1,1]:.3f}")

```{admonition} Exercise 1: Building a Large Sparse Network
:class: dropdown

Create a radial distribution network with 100 buses:
1. Each bus connects to the next bus (bus i to bus i+1)
2. Every 10th bus has a lateral connection to bus i+5
3. Use random impedances: R between 0.01-0.05, X between 0.03-0.10
4. Build the Y-bus matrix and analyze its sparsity
5. Compare memory usage with a dense matrix

```

In [ ]:
# Solution to Exercise 1
n_buses = 100

# Generate branch data
branches = []

# Main feeder connections
for i in range(n_buses - 1):
    r = np.random.uniform(0.01, 0.05)
    x = np.random.uniform(0.03, 0.10)
    z = r + 1j*x
    y = 1/z  # Admittance
    branches.append((i, i+1, y))

# Lateral connections
for i in range(0, n_buses-5, 10):
    r = np.random.uniform(0.01, 0.05)
    x = np.random.uniform(0.03, 0.10)
    z = r + 1j*x
    y = 1/z
    branches.append((i, i+5, y))

print(f"Distribution Network:")
print(f"Number of buses: {n_buses}")
print(f"Number of branches: {len(branches)}")

# Build Y-bus matrix
ybus = sp.dok_matrix((n_buses, n_buses), dtype=complex)

for from_bus, to_bus, y in branches:
    ybus[from_bus, to_bus] -= y
    ybus[to_bus, from_bus] -= y
    ybus[from_bus, from_bus] += y
    ybus[to_bus, to_bus] += y

# Convert to CSR
ybus_csr = ybus.tocsr()

# Analyze sparsity
nnz = ybus_csr.nnz
total_elements = n_buses * n_buses
sparsity = nnz / total_elements * 100

print(f"\nSparsity Analysis:")
print(f"Non-zero elements: {nnz}")
print(f"Total elements: {total_elements}")
print(f"Sparsity: {sparsity:.2f}%")
print(f"Average connections per bus: {nnz / n_buses:.1f}")

# Memory comparison
# Dense matrix
dense_size = n_buses * n_buses * 16  # 16 bytes for complex128
# Sparse matrix (approximate)
sparse_size = nnz * 16 + nnz * 4 + (n_buses + 1) * 4  # data + indices + indptr

print(f"\nMemory Usage:")
print(f"Dense matrix: {dense_size / 1024:.1f} KB")
print(f"Sparse matrix: {sparse_size / 1024:.1f} KB")
print(f"Memory savings: {(1 - sparse_size/dense_size)*100:.1f}%")

# Visualize sparsity pattern
plt.figure(figsize=(8, 8))
plt.spy(ybus_csr, markersize=1)
plt.title(f'Distribution Network Y-bus Sparsity Pattern ({sparsity:.1f}% sparse)')
plt.xlabel('Bus Index')
plt.ylabel('Bus Index')
plt.show()

## 2. Sparse Matrix Operations

Efficient operations on sparse matrices are crucial for power system analysis. Let's explore common operations and their performance implications.

### Matrix-Vector Multiplication

In [ ]:
# Compare sparse vs dense matrix-vector multiplication
# This is fundamental for power flow iterations

# Create a larger test case
n = 500
density = 0.01  # 1% non-zero elements

# Generate random sparse matrix
A_sparse = sp.random(n, n, density=density, format='csr')
A_dense = A_sparse.toarray()

# Random vector
x = np.random.randn(n)

# Time sparse multiplication
start = time.time()
for _ in range(100):
    y_sparse = A_sparse @ x
sparse_time = time.time() - start

# Time dense multiplication
start = time.time()
for _ in range(100):
    y_dense = A_dense @ x
dense_time = time.time() - start

print(f"Matrix-Vector Multiplication ({n}×{n} matrix, {density*100:.1f}% density):")
print(f"Sparse: {sparse_time*1000:.2f} ms")
print(f"Dense: {dense_time*1000:.2f} ms")
print(f"Speedup: {dense_time/sparse_time:.1f}x")
print(f"\nResults match: {np.allclose(y_sparse, y_dense)}")

# Analyze operation count
flops_dense = 2 * n * n  # Multiplications and additions
flops_sparse = 2 * A_sparse.nnz
print(f"\nOperation count:")
print(f"Dense: {flops_dense:,} FLOPs")
print(f"Sparse: {flops_sparse:,} FLOPs")
print(f"Reduction: {(1 - flops_sparse/flops_dense)*100:.1f}%")

### Solving Linear Systems

In [ ]:
# Solving Ax = b with sparse matrices
# Essential for power flow and state estimation

# Create a symmetric positive definite sparse matrix
# (like those in power system optimization)
n = 200
# Create a random sparse matrix
A_temp = sp.random(n, n, density=0.05, format='csr')
# Make it symmetric positive definite
A_sparse = A_temp @ A_temp.T + sp.eye(n) * 10
A_dense = A_sparse.toarray()

# Right-hand side
b = np.random.randn(n)

# Method 1: Direct sparse solver
start = time.time()
x_sparse_direct = sp_linalg.spsolve(A_sparse, b)
time_sparse_direct = time.time() - start

# Method 2: Sparse LU decomposition
start = time.time()
lu = sp_linalg.splu(A_sparse.tocsc())
x_sparse_lu = lu.solve(b)
time_sparse_lu = time.time() - start

# Method 3: Dense solver (for comparison)
start = time.time()
x_dense = np.linalg.solve(A_dense, b)
time_dense = time.time() - start

print(f"Solving {n}×{n} linear system:")
print(f"Sparse direct: {time_sparse_direct*1000:.2f} ms")
print(f"Sparse LU: {time_sparse_lu*1000:.2f} ms")
print(f"Dense: {time_dense*1000:.2f} ms")
print(f"\nSpeedup (sparse direct vs dense): {time_dense/time_sparse_direct:.1f}x")

# Verify solutions
print(f"\nSolution verification:")
print(f"Sparse direct vs dense: {np.allclose(x_sparse_direct, x_dense)}")
print(f"Sparse LU vs dense: {np.allclose(x_sparse_lu, x_dense)}")

# Residual analysis
residual_sparse = np.linalg.norm(A_sparse @ x_sparse_direct - b)
residual_dense = np.linalg.norm(A_dense @ x_dense - b)
print(f"\nResiduals:")
print(f"Sparse: {residual_sparse:.2e}")
print(f"Dense: {residual_dense:.2e}")

### Iterative Solvers

In [ ]:
# Iterative methods for very large systems
# Used when direct methods become impractical

# Create a larger system
n = 1000
# Diagonally dominant sparse matrix (convergence guaranteed)
A = sp.random(n, n, density=0.01, format='csr')
A = A @ A.T  # Make symmetric
A = A + sp.eye(n) * (np.abs(A).sum(axis=1).max() + 1)  # Ensure diagonal dominance

b = np.random.randn(n)

# Compare different iterative methods
methods = [
    ('CG', sp_linalg.cg),         # Conjugate Gradient
    ('GMRES', sp_linalg.gmres),   # Generalized Minimal Residual
    ('BiCGSTAB', sp_linalg.bicgstab)  # Bi-Conjugate Gradient Stabilized
]

print(f"Iterative Solvers for {n}×{n} system:")
print(f"Matrix density: {A.nnz / (n*n) * 100:.2f}%\n")

x_ref = sp_linalg.spsolve(A, b)  # Reference solution

for name, solver in methods:
    start = time.time()
    x, info = solver(A, b, tol=1e-8, maxiter=100)
    elapsed = time.time() - start
    
    if info == 0:
        error = np.linalg.norm(x - x_ref) / np.linalg.norm(x_ref)
        residual = np.linalg.norm(A @ x - b)
        print(f"{name:10s}: {elapsed*1000:6.2f} ms, error: {error:.2e}, residual: {residual:.2e}")
    else:
        print(f"{name:10s}: Failed to converge (info={info})")

# Demonstrate preconditioning effect
print("\nWith Preconditioning:")
# Simple diagonal preconditioner
M = sp.diags(1.0 / A.diagonal())

start = time.time()
x_precond, info = sp_linalg.cg(A, b, M=M, tol=1e-8, maxiter=100)
elapsed_precond = time.time() - start

if info == 0:
    error = np.linalg.norm(x_precond - x_ref) / np.linalg.norm(x_ref)
    print(f"CG with preconditioner: {elapsed_precond*1000:.2f} ms, error: {error:.2e}")

```{admonition} Exercise 2: Sparse System Solution
:class: dropdown

Create and solve a sparse linear system representing a DC power flow problem:
1. Create a 50-bus system with random connections (average 3 per bus)
2. Build the B matrix (susceptance matrix) with random values between 5-20 p.u.
3. Create power injection vector with random values between -1 and 1 p.u.
4. Solve for bus angles using both direct and iterative methods
5. Compare computation times and verify results

```

In [ ]:
# Solution to Exercise 2
np.random.seed(123)
n_buses = 50
avg_connections = 3

# Create random connections
connections = set()
for i in range(n_buses):
    # Connect to random buses
    n_conn = np.random.poisson(avg_connections)
    for _ in range(n_conn):
        j = np.random.randint(0, n_buses)
        if i != j:
            connections.add((min(i,j), max(i,j)))

connections = list(connections)
print(f"DC Power Flow Problem:")
print(f"Buses: {n_buses}")
print(f"Lines: {len(connections)}")

# Build B matrix
B = sp.dok_matrix((n_buses, n_buses))

for i, j in connections:
    b_ij = np.random.uniform(5, 20)  # Susceptance
    B[i, j] -= b_ij
    B[j, i] -= b_ij
    B[i, i] += b_ij
    B[j, j] += b_ij

B = B.tocsr()

# Power injections (exclude slack bus)
P = np.random.uniform(-1, 1, n_buses)
P[0] = 0  # Slack bus

# Remove slack bus from system
B_reduced = B[1:, 1:]
P_reduced = P[1:]

print(f"\nSystem matrix: {B_reduced.shape}")
print(f"Sparsity: {B_reduced.nnz / (B_reduced.shape[0]**2) * 100:.1f}%")

# Method 1: Direct solver
start = time.time()
theta_direct = sp_linalg.spsolve(B_reduced, P_reduced)
time_direct = time.time() - start

# Method 2: CG solver
start = time.time()
theta_cg, info_cg = sp_linalg.cg(B_reduced, P_reduced, tol=1e-10)
time_cg = time.time() - start

# Method 3: LU decomposition
start = time.time()
lu = sp_linalg.splu(B_reduced.tocsc())
theta_lu = lu.solve(P_reduced)
time_lu = time.time() - start

print(f"\nSolution times:")
print(f"Direct: {time_direct*1000:.3f} ms")
print(f"CG: {time_cg*1000:.3f} ms (converged: {info_cg == 0})")
print(f"LU: {time_lu*1000:.3f} ms")

# Verify solutions
print(f"\nSolution verification:")
print(f"Direct vs CG: max diff = {np.max(np.abs(theta_direct - theta_cg)):.2e}")
print(f"Direct vs LU: max diff = {np.max(np.abs(theta_direct - theta_lu)):.2e}")

# Check power balance
theta_full = np.zeros(n_buses)
theta_full[1:] = theta_direct

# Calculate flows and verify power balance
P_calc = B @ theta_full
P[0] = P_calc[0]  # Slack bus power

print(f"\nPower balance check:")
print(f"Total generation: {P[P>0].sum():.3f} p.u.")
print(f"Total load: {-P[P<0].sum():.3f} p.u.")
print(f"Mismatch: {P.sum():.2e} p.u.")

## 3. Advanced Linear Algebra

Module 04 requires understanding of matrix factorizations and numerical stability. Let's explore these concepts with power system examples.

### LU Factorization

In [ ]:
# LU factorization for repeated solutions
# Common in power flow where we solve with same matrix, different RHS

# Create a test matrix (simplified Y-bus)
n = 10
Y = sp.random(n, n, density=0.3, format='csr')
Y = Y + Y.T  # Make symmetric
Y = Y + sp.eye(n) * 5  # Ensure non-singular

# Multiple right-hand sides (different current injections)
n_cases = 100
I_injections = np.random.randn(n, n_cases) + 1j * np.random.randn(n, n_cases)

print("Solving multiple right-hand sides:")
print(f"Matrix size: {n}×{n}")
print(f"Number of RHS vectors: {n_cases}\n")

# Method 1: Solve each independently
start = time.time()
V_independent = np.zeros((n, n_cases), dtype=complex)
for i in range(n_cases):
    V_independent[:, i] = sp_linalg.spsolve(Y, I_injections[:, i])
time_independent = time.time() - start

# Method 2: LU factorization once, solve multiple times
start = time.time()
lu = sp_linalg.splu(Y.tocsc())
time_factor = time.time() - start

start = time.time()
V_lu = np.zeros((n, n_cases), dtype=complex)
for i in range(n_cases):
    V_lu[:, i] = lu.solve(I_injections[:, i])
time_solve = time.time() - start

print(f"Independent solutions: {time_independent*1000:.1f} ms")
print(f"LU factorization: {time_factor*1000:.1f} ms")
print(f"LU back-substitution: {time_solve*1000:.1f} ms")
print(f"Total LU time: {(time_factor + time_solve)*1000:.1f} ms")
print(f"\nSpeedup: {time_independent/(time_factor + time_solve):.1f}x")

# Verify results
print(f"\nResults match: {np.allclose(V_independent, V_lu)}")

# Analyze LU factors
print(f"\nLU factorization analysis:")
print(f"Original matrix non-zeros: {Y.nnz}")
print(f"L factor non-zeros: {lu.L.nnz}")
print(f"U factor non-zeros: {lu.U.nnz}")
print(f"Fill-in: {(lu.L.nnz + lu.U.nnz - Y.nnz) / Y.nnz * 100:.1f}%")

### Matrix Conditioning and Numerical Stability

In [ ]:
# Understanding condition numbers and their impact
# Important for power flow convergence

# Create matrices with different condition numbers
n = 50

# Well-conditioned matrix
A_good = sp.random(n, n, density=0.1)
A_good = A_good @ A_good.T + sp.eye(n) * 10

# Poorly-conditioned matrix
A_bad = sp.random(n, n, density=0.1)
A_bad = A_bad @ A_bad.T + sp.eye(n) * 0.001

# Calculate condition numbers
cond_good = np.linalg.cond(A_good.toarray())
cond_bad = np.linalg.cond(A_bad.toarray())

print("Matrix Conditioning Analysis:")
print(f"Well-conditioned matrix: κ = {cond_good:.2e}")
print(f"Poorly-conditioned matrix: κ = {cond_bad:.2e}")

# Test sensitivity to perturbations
b = np.random.randn(n)
x_good = sp_linalg.spsolve(A_good, b)
x_bad = sp_linalg.spsolve(A_bad, b)

# Add small perturbation to b
delta_b = np.random.randn(n) * 1e-6
b_perturbed = b + delta_b

x_good_pert = sp_linalg.spsolve(A_good, b_perturbed)
x_bad_pert = sp_linalg.spsolve(A_bad, b_perturbed)

# Calculate relative changes
rel_change_b = np.linalg.norm(delta_b) / np.linalg.norm(b)
rel_change_x_good = np.linalg.norm(x_good_pert - x_good) / np.linalg.norm(x_good)
rel_change_x_bad = np.linalg.norm(x_bad_pert - x_bad) / np.linalg.norm(x_bad)

print(f"\nPerturbation Analysis:")
print(f"Relative change in b: {rel_change_b:.2e}")
print(f"Relative change in x (good): {rel_change_x_good:.2e}")
print(f"Relative change in x (bad): {rel_change_x_bad:.2e}")
print(f"\nAmplification factor (good): {rel_change_x_good/rel_change_b:.1f}")
print(f"Amplification factor (bad): {rel_change_x_bad/rel_change_b:.1f}")

# Iterative refinement for ill-conditioned systems
def iterative_refinement(A, b, x0, max_iter=5):
    """Improve solution accuracy through iterative refinement"""
    x = x0.copy()
    for i in range(max_iter):
        r = b - A @ x
        delta_x = sp_linalg.spsolve(A, r)
        x = x + delta_x
        if np.linalg.norm(delta_x) < 1e-10:
            break
    return x, i+1

# Apply to poorly-conditioned system
x_refined, iterations = iterative_refinement(A_bad, b, x_bad)
residual_original = np.linalg.norm(A_bad @ x_bad - b)
residual_refined = np.linalg.norm(A_bad @ x_refined - b)

print(f"\nIterative Refinement:")
print(f"Original residual: {residual_original:.2e}")
print(f"Refined residual: {residual_refined:.2e}")
print(f"Improvement factor: {residual_original/residual_refined:.1f}")
print(f"Iterations: {iterations}")

### Eigenvalue Analysis

In [ ]:
# Eigenvalue analysis for system stability
# Used in small-signal stability studies

# Create a state matrix for a simple power system model
# 2-state model per generator: angle and speed
n_gen = 3
n_states = 2 * n_gen

# System parameters
H = np.array([5.0, 4.0, 3.0])  # Inertia constants
D = np.array([1.0, 0.8, 1.2])  # Damping coefficients

# Simplified state matrix (demonstration)
A = np.zeros((n_states, n_states))

# Fill state matrix
for i in range(n_gen):
    # Speed-angle relationship
    A[2*i, 2*i+1] = 377.0  # 2*pi*60 rad/s
    # Damping
    A[2*i+1, 2*i+1] = -D[i] / (2*H[i])
    # Synchronizing torque (simplified)
    A[2*i+1, 2*i] = -1.0 / (2*H[i])

# Add coupling between generators
coupling = 0.5
for i in range(n_gen):
    for j in range(n_gen):
        if i != j:
            A[2*i+1, 2*j] = coupling / (2*H[i])

print("Small-Signal Stability Analysis")
print(f"System: {n_gen} generators, {n_states} states\n")

# Calculate eigenvalues
eigenvalues, eigenvectors = np.linalg.eig(A)

# Analyze stability
print("Eigenvalue Analysis:")
print("Mode   Real Part   Imag Part   Frequency(Hz)   Damping(%)   Stability")
print("=" * 70)

for i, eigval in enumerate(eigenvalues):
    real_part = eigval.real
    imag_part = eigval.imag
    
    if abs(imag_part) > 1e-6:  # Oscillatory mode
        frequency = abs(imag_part) / (2 * np.pi)
        damping = -real_part / abs(eigval) * 100
        stability = "Stable" if real_part < 0 else "UNSTABLE"
    else:  # Non-oscillatory mode
        frequency = 0
        damping = 100 if real_part < 0 else 0
        stability = "Stable" if real_part < 0 else "UNSTABLE"
    
    print(f"{i+1:4d}   {real_part:9.4f}   {imag_part:9.4f}   {frequency:11.3f}   "
          f"{damping:9.1f}   {stability}")

# Participation factors
print("\nParticipation Factors (dominant modes):")
# Find oscillatory modes
osc_modes = np.where(np.abs(eigenvalues.imag) > 1e-6)[0]

if len(osc_modes) > 0:
    mode = osc_modes[0]  # First oscillatory mode
    eigvec = eigenvectors[:, mode]
    
    # Normalize by maximum
    participation = np.abs(eigvec) / np.max(np.abs(eigvec))
    
    print(f"\nMode {mode+1} participation:")
    for i in range(n_gen):
        angle_part = participation[2*i]
        speed_part = participation[2*i+1]
        print(f"Generator {i+1}: angle={angle_part:.3f}, speed={speed_part:.3f}")

```{admonition} Exercise 3: Eigenvalue Sensitivity
:class: dropdown

Analyze how system parameters affect eigenvalues:
1. Create a 2×2 system matrix representing a single-machine infinite bus
2. Vary the damping coefficient from 0.5 to 2.0
3. Calculate eigenvalues for each damping value
4. Plot the eigenvalue trajectories in the complex plane
5. Identify the critical damping value

```

In [ ]:
# Solution to Exercise 3
# Single machine infinite bus model
# States: [delta, omega] (angle and speed deviation)

# Fixed parameters
H = 5.0  # Inertia constant
omega_s = 377.0  # Synchronous speed (rad/s)
P_max = 1.0  # Maximum power transfer
delta_0 = np.pi/6  # Initial operating angle (30 degrees)

# Linearized synchronizing torque coefficient
K_s = P_max * np.cos(delta_0)

# Vary damping
D_values = np.linspace(0.5, 2.0, 50)
eigenvalues_all = []

for D in D_values:
    # State matrix
    A = np.array([
        [0, omega_s],
        [-K_s/(2*H), -D/(2*H)]
    ])
    
    eigvals = np.linalg.eigvals(A)
    eigenvalues_all.append(eigvals)

eigenvalues_all = np.array(eigenvalues_all)

# Plot eigenvalue trajectories
plt.figure(figsize=(10, 8))

# Plot in complex plane
plt.subplot(2, 2, 1)
plt.plot(eigenvalues_all[:, 0].real, eigenvalues_all[:, 0].imag, 'b-', linewidth=2)
plt.plot(eigenvalues_all[:, 1].real, eigenvalues_all[:, 1].imag, 'r-', linewidth=2)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
plt.xlabel('Real Part')
plt.ylabel('Imaginary Part')
plt.title('Eigenvalue Trajectories')
plt.grid(True, alpha=0.3)

# Damping ratio vs D
plt.subplot(2, 2, 2)
damping_ratios = []
frequencies = []

for eigvals in eigenvalues_all:
    if np.abs(eigvals[0].imag) > 1e-6:  # Oscillatory
        damping_ratio = -eigvals[0].real / np.abs(eigvals[0])
        frequency = np.abs(eigvals[0].imag) / (2 * np.pi)
    else:  # Critically or over-damped
        damping_ratio = 1.0
        frequency = 0
    damping_ratios.append(damping_ratio)
    frequencies.append(frequency)

plt.plot(D_values, damping_ratios, 'g-', linewidth=2)
plt.axhline(y=0.707, color='r', linestyle='--', label='ζ = 0.707')
plt.xlabel('Damping Coefficient D')
plt.ylabel('Damping Ratio ζ')
plt.title('Damping Ratio vs Damping Coefficient')
plt.grid(True, alpha=0.3)
plt.legend()

# Frequency vs D
plt.subplot(2, 2, 3)
plt.plot(D_values, frequencies, 'm-', linewidth=2)
plt.xlabel('Damping Coefficient D')
plt.ylabel('Oscillation Frequency (Hz)')
plt.title('Natural Frequency vs Damping')
plt.grid(True, alpha=0.3)

# Real parts vs D
plt.subplot(2, 2, 4)
plt.plot(D_values, eigenvalues_all[:, 0].real, 'b-', linewidth=2, label='λ₁')
plt.plot(D_values, eigenvalues_all[:, 1].real, 'r-', linewidth=2, label='λ₂')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.xlabel('Damping Coefficient D')
plt.ylabel('Real Part of Eigenvalue')
plt.title('Stability vs Damping')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

# Find critical damping
critical_idx = np.where(np.array(frequencies) < 0.01)[0]
if len(critical_idx) > 0:
    D_critical = D_values[critical_idx[0]]
    print(f"Critical damping occurs at D ≈ {D_critical:.3f}")
    
    # Theoretical critical damping
    D_critical_theory = 2 * np.sqrt(2 * H * K_s)
    print(f"Theoretical critical damping: D = {D_critical_theory:.3f}")
    print(f"Error: {abs(D_critical - D_critical_theory):.3f}")

## 4. Performance Optimization

Efficient NumPy code is crucial for large-scale power system analysis. Let's explore optimization techniques.

### Memory Layout and Cache Efficiency

In [ ]:
# Understanding memory layout impact on performance
n = 1000
m = 1000

# Create test matrices
A = np.random.randn(n, m)
B = np.random.randn(m, n)

# Row-major (C-order) vs Column-major (Fortran-order)
A_C = np.ascontiguousarray(A)  # C-order
A_F = np.asfortranarray(A)     # Fortran-order

print("Memory Layout Performance:")
print(f"Matrix shape: {A.shape}")

# Test 1: Row-wise operations
print("\n1. Row-wise sum:")
# C-order (row-major)
start = time.time()
for _ in range(10):
    row_sum_C = A_C.sum(axis=1)
time_C = time.time() - start

# F-order (column-major)
start = time.time()
for _ in range(10):
    row_sum_F = A_F.sum(axis=1)
time_F = time.time() - start

print(f"   C-order: {time_C*1000:.2f} ms")
print(f"   F-order: {time_F*1000:.2f} ms")
print(f"   Speedup: {time_F/time_C:.2f}x")

# Test 2: Column-wise operations
print("\n2. Column-wise sum:")
# C-order
start = time.time()
for _ in range(10):
    col_sum_C = A_C.sum(axis=0)
time_C = time.time() - start

# F-order
start = time.time()
for _ in range(10):
    col_sum_F = A_F.sum(axis=0)
time_F = time.time() - start

print(f"   C-order: {time_C*1000:.2f} ms")
print(f"   F-order: {time_F*1000:.2f} ms")
print(f"   Speedup: {time_C/time_F:.2f}x")

# Test 3: Matrix multiplication
print("\n3. Matrix multiplication (A @ B):")
# Default
start = time.time()
C1 = A @ B
time_default = time.time() - start

# Optimized order
start = time.time()
C2 = np.ascontiguousarray(A) @ np.asfortranarray(B)
time_optimized = time.time() - start

print(f"   Default: {time_default*1000:.2f} ms")
print(f"   Optimized: {time_optimized*1000:.2f} ms")
print(f"   Speedup: {time_default/time_optimized:.2f}x")

### Vectorization Strategies

In [ ]:
# Vectorizing power system calculations
# Example: Calculate power losses for all lines

# Network data
n_lines = 1000
V_from = np.random.uniform(0.95, 1.05, n_lines) * np.exp(1j * np.random.uniform(-0.1, 0.1, n_lines))
V_to = np.random.uniform(0.95, 1.05, n_lines) * np.exp(1j * np.random.uniform(-0.1, 0.1, n_lines))
Y_lines = np.random.uniform(5, 20, n_lines) - 1j * np.random.uniform(15, 60, n_lines)

print(f"Power Loss Calculation for {n_lines} lines:")

# Method 1: Loop-based (slow)
def calculate_losses_loop(V_from, V_to, Y_lines):
    losses = np.zeros(len(V_from))
    for i in range(len(V_from)):
        I = (V_from[i] - V_to[i]) * Y_lines[i]
        losses[i] = np.real(I * np.conj(I) / Y_lines[i])
    return losses

# Method 2: Vectorized
def calculate_losses_vectorized(V_from, V_to, Y_lines):
    I = (V_from - V_to) * Y_lines
    return np.real(I * np.conj(I) / Y_lines)

# Method 3: Optimized vectorized
def calculate_losses_optimized(V_from, V_to, Y_lines):
    V_diff = V_from - V_to
    I_squared = np.abs(V_diff)**2 * np.abs(Y_lines)**2
    return I_squared / np.abs(Y_lines)

# Time comparison
start = time.time()
losses_loop = calculate_losses_loop(V_from, V_to, Y_lines)
time_loop = time.time() - start

start = time.time()
losses_vec = calculate_losses_vectorized(V_from, V_to, Y_lines)
time_vec = time.time() - start

start = time.time()
losses_opt = calculate_losses_optimized(V_from, V_to, Y_lines)
time_opt = time.time() - start

print(f"\nLoop-based: {time_loop*1000:.2f} ms")
print(f"Vectorized: {time_vec*1000:.2f} ms (speedup: {time_loop/time_vec:.1f}x)")
print(f"Optimized: {time_opt*1000:.2f} ms (speedup: {time_loop/time_opt:.1f}x)")

# Verify results
print(f"\nResults match:")
print(f"Loop vs vectorized: {np.allclose(losses_loop, losses_vec)}")
print(f"Loop vs optimized: {np.allclose(losses_loop, losses_opt, rtol=1e-5)}")

# Total system losses
print(f"\nTotal system losses: {np.sum(losses_vec):.2f} MW")

### Using NumPy's Built-in Functions

In [ ]:
# Leveraging optimized NumPy functions
# Example: Finding constrained generators

# Generator data
n_gen = 100
P_min = np.random.uniform(10, 50, n_gen)
P_max = P_min + np.random.uniform(50, 200, n_gen)
P_current = np.random.uniform(0, 1, n_gen) * (P_max - P_min) + P_min

# Add some generators at limits
P_current[::10] = P_min[::10]  # Every 10th at min
P_current[5::10] = P_max[5::10]  # Every 10th at max

print("Finding Constrained Generators:")

# Method 1: Loop with conditions
def find_constrained_loop(P_current, P_min, P_max, tol=1e-6):
    at_min = []
    at_max = []
    unconstrained = []
    
    for i in range(len(P_current)):
        if abs(P_current[i] - P_min[i]) < tol:
            at_min.append(i)
        elif abs(P_current[i] - P_max[i]) < tol:
            at_max.append(i)
        else:
            unconstrained.append(i)
    
    return at_min, at_max, unconstrained

# Method 2: NumPy boolean indexing
def find_constrained_numpy(P_current, P_min, P_max, tol=1e-6):
    at_min = np.where(np.abs(P_current - P_min) < tol)[0]
    at_max = np.where(np.abs(P_current - P_max) < tol)[0]
    constrained = np.concatenate([at_min, at_max])
    unconstrained = np.setdiff1d(np.arange(len(P_current)), constrained)
    
    return at_min, at_max, unconstrained

# Time comparison
n_runs = 1000

start = time.time()
for _ in range(n_runs):
    min1, max1, free1 = find_constrained_loop(P_current, P_min, P_max)
time_loop = time.time() - start

start = time.time()
for _ in range(n_runs):
    min2, max2, free2 = find_constrained_numpy(P_current, P_min, P_max)
time_numpy = time.time() - start

print(f"\nLoop method: {time_loop*1000:.2f} ms")
print(f"NumPy method: {time_numpy*1000:.2f} ms")
print(f"Speedup: {time_loop/time_numpy:.1f}x")

print(f"\nResults:")
print(f"Generators at min: {len(min2)}")
print(f"Generators at max: {len(max2)}")
print(f"Unconstrained: {len(free2)}")

# Additional analysis using NumPy functions
margin_to_max = P_max - P_current
margin_to_min = P_current - P_min

print(f"\nMargin Analysis:")
print(f"Average upward margin: {np.mean(margin_to_max):.1f} MW")
print(f"Average downward margin: {np.mean(margin_to_min):.1f} MW")
print(f"Total upward reserve: {np.sum(margin_to_max):.0f} MW")
print(f"Total downward reserve: {np.sum(margin_to_min):.0f} MW")

# Generators close to limits
threshold = 5.0  # MW
near_max = np.sum(margin_to_max < threshold)
near_min = np.sum(margin_to_min < threshold)
print(f"\nGenerators within {threshold} MW of limits:")
print(f"Near max: {near_max}")
print(f"Near min: {near_min}")

```{admonition} Exercise 4: Performance Optimization Challenge
:class: dropdown

Optimize a power flow mismatch calculation:
1. Create a 100-bus system with random Y-bus (5% density)
2. Generate random voltage magnitudes and angles
3. Calculate power mismatches P_specified - P_calculated
4. Implement three versions: loop-based, basic vectorized, optimized
5. Compare performance and verify correctness

```

In [ ]:
# Solution to Exercise 4
n = 100
density = 0.05

# Create Y-bus
Y_sparse = sp.random(n, n, density=density, dtype=complex)
Y_sparse = Y_sparse + Y_sparse.T  # Make symmetric
# Make diagonally dominant
for i in range(n):
    Y_sparse[i, i] = np.sum(np.abs(Y_sparse[i, :])) + 1

Y_dense = Y_sparse.toarray()

# Voltage data
V_mag = np.random.uniform(0.95, 1.05, n)
V_angle = np.random.uniform(-0.2, 0.2, n)
V_complex = V_mag * np.exp(1j * V_angle)

# Specified power
P_spec = np.random.uniform(-2, 2, n)
P_spec[0] = 0  # Slack bus

print("Power Flow Mismatch Calculation:")
print(f"System: {n} buses, {Y_sparse.nnz} non-zero Y-bus elements\n")

# Method 1: Loop-based
def mismatch_loop(V_complex, Y, P_spec):
    n = len(V_complex)
    P_calc = np.zeros(n)
    
    for i in range(n):
        for j in range(n):
            if Y[i, j] != 0:
                P_calc[i] += np.real(V_complex[i] * np.conj(Y[i, j] * V_complex[j]))
    
    return P_spec - P_calc

# Method 2: Basic vectorized
def mismatch_vectorized(V_complex, Y, P_spec):
    S_calc = V_complex * np.conj(Y @ V_complex)
    P_calc = S_calc.real
    return P_spec - P_calc

# Method 3: Optimized with sparse
def mismatch_optimized(V_complex, Y_sparse, P_spec):
    # Use sparse matrix multiplication
    I_calc = Y_sparse @ V_complex
    S_calc = V_complex * np.conj(I_calc)
    return P_spec - S_calc.real

# Time comparison
n_runs = 10

# Loop method (only run once due to slowness)
start = time.time()
mismatch1 = mismatch_loop(V_complex, Y_dense, P_spec)
time_loop = time.time() - start

# Vectorized method
start = time.time()
for _ in range(n_runs):
    mismatch2 = mismatch_vectorized(V_complex, Y_dense, P_spec)
time_vec = (time.time() - start) / n_runs

# Optimized method
start = time.time()
for _ in range(n_runs):
    mismatch3 = mismatch_optimized(V_complex, Y_sparse, P_spec)
time_opt = (time.time() - start) / n_runs

print(f"Performance Comparison:")
print(f"Loop-based: {time_loop*1000:.2f} ms")
print(f"Vectorized: {time_vec*1000:.2f} ms (speedup: {time_loop/time_vec:.1f}x)")
print(f"Optimized: {time_opt*1000:.2f} ms (speedup: {time_loop/time_opt:.1f}x)")

# Verify correctness
print(f"\nVerification:")
print(f"Loop vs vectorized: max diff = {np.max(np.abs(mismatch1 - mismatch2)):.2e}")
print(f"Loop vs optimized: max diff = {np.max(np.abs(mismatch1 - mismatch3)):.2e}")

# Analyze mismatches
print(f"\nMismatch Statistics:")
print(f"Maximum mismatch: {np.max(np.abs(mismatch3)):.3f} p.u.")
print(f"Average mismatch: {np.mean(np.abs(mismatch3)):.3f} p.u.")
print(f"RMS mismatch: {np.sqrt(np.mean(mismatch3**2)):.3f} p.u.")

# Memory usage comparison
print(f"\nMemory Usage:")
print(f"Dense Y-bus: {Y_dense.nbytes / 1024:.1f} KB")
print(f"Sparse Y-bus: {(Y_sparse.data.nbytes + Y_sparse.indices.nbytes + Y_sparse.indptr.nbytes) / 1024:.1f} KB")
print(f"Memory savings: {(1 - (Y_sparse.data.nbytes + Y_sparse.indices.nbytes + Y_sparse.indptr.nbytes) / Y_dense.nbytes) * 100:.1f}%")

## 5. Integration with Module 04

Let's explore specific techniques that directly prepare you for Module 04's numerical methods.

### Jacobian Matrix Construction

In [ ]:
# Building Jacobian matrices for Newton-Raphson
# Essential for Module 04's power flow implementation

def build_jacobian_template(n_buses, bus_types, Y_sparse):
    """
    Build Jacobian matrix structure for power flow.
    bus_types: 0=slack, 1=PV, 2=PQ
    """
    # Identify bus indices
    pq_buses = np.where(bus_types == 2)[0]
    pv_buses = np.where(bus_types == 1)[0]
    non_slack = np.concatenate([pv_buses, pq_buses])
    
    n_pq = len(pq_buses)
    n_pv = len(pv_buses)
    
    # Jacobian sub-matrices dimensions
    # J = [J11 J12]  where J11: dP/dθ, J12: dP/dV
    #     [J21 J22]        J21: dQ/dθ, J22: dQ/dV
    
    print(f"Jacobian Structure:")
    print(f"PV buses: {n_pv}, PQ buses: {n_pq}")
    print(f"J11: {n_pv + n_pq} × {n_pv + n_pq} (dP/dθ)")
    print(f"J12: {n_pv + n_pq} × {n_pq} (dP/dV)")
    print(f"J21: {n_pq} × {n_pv + n_pq} (dQ/dθ)")
    print(f"J22: {n_pq} × {n_pq} (dQ/dV)")
    
    # Determine sparsity pattern from Y-bus
    # Jacobian element (i,j) is non-zero if Y[i,j] is non-zero
    rows = []
    cols = []
    
    # J11 pattern (all non-slack buses)
    for i_idx, i in enumerate(non_slack):
        for j_idx, j in enumerate(non_slack):
            if Y_sparse[i, j] != 0:
                rows.append(i_idx)
                cols.append(j_idx)
    
    n_jacobian = len(non_slack) + n_pq
    J_pattern = sp.csr_matrix(
        (np.ones(len(rows)), (rows, cols)),
        shape=(len(non_slack), len(non_slack))
    )
    
    return J_pattern, non_slack, pq_buses

# Example system
n = 10
bus_types = np.array([0, 1, 1, 2, 2, 2, 2, 2, 2, 2])  # 1 slack, 2 PV, 7 PQ

# Create sparse Y-bus
Y_bus = sp.random(n, n, density=0.3, dtype=complex)
Y_bus = Y_bus + Y_bus.T

J_pattern, non_slack, pq_buses = build_jacobian_template(n, bus_types, Y_bus)

print(f"\nJacobian sparsity: {J_pattern.nnz / (J_pattern.shape[0]**2) * 100:.1f}%")

# Visualize Jacobian structure
plt.figure(figsize=(8, 8))
plt.spy(J_pattern, markersize=5)
plt.title('Jacobian Sparsity Pattern')
plt.xlabel('Column Index')
plt.ylabel('Row Index')

# Add grid to show sub-matrices
n_pv = np.sum(bus_types == 1)
n_pq = np.sum(bus_types == 2)
plt.axhline(y=n_pv + n_pq - 0.5, color='r', linewidth=2)
plt.axvline(x=n_pv + n_pq - 0.5, color='r', linewidth=2)
plt.text(2, 2, 'J11', fontsize=12, color='blue')
plt.text(n_pv + n_pq + 1, 2, 'J12', fontsize=12, color='blue')
plt.text(2, n_pv + n_pq + 1, 'J21', fontsize=12, color='blue')
plt.text(n_pv + n_pq + 1, n_pv + n_pq + 1, 'J22', fontsize=12, color='blue')

plt.show()

### Preparing for Numerical Methods

In [ ]:
# Key numerical concepts for Module 04

# 1. Matrix reordering for sparsity preservation
from scipy.sparse import csgraph

# Create a sample sparse matrix
n = 20
A = sp.random(n, n, density=0.2, format='csr')
A = A + A.T + sp.eye(n) * 5

# Find reordering to minimize fill-in
perm = csgraph.reverse_cuthill_mckee(A)
A_reordered = A[perm][:, perm]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

ax1.spy(A, markersize=3)
ax1.set_title('Original Matrix')

ax2.spy(A_reordered, markersize=3)
ax2.set_title('Reordered Matrix (RCM)')

plt.tight_layout()
plt.show()

# 2. Convergence monitoring for iterative methods
def power_iteration_with_monitoring(A, max_iter=100, tol=1e-6):
    """Power iteration to find dominant eigenvalue"""
    n = A.shape[0]
    x = np.random.randn(n)
    x = x / np.linalg.norm(x)
    
    convergence_history = []
    eigenvalue_history = []
    
    for i in range(max_iter):
        x_new = A @ x
        eigenvalue = np.dot(x, x_new)
        x_new = x_new / np.linalg.norm(x_new)
        
        # Convergence metric
        convergence = np.linalg.norm(x_new - x)
        convergence_history.append(convergence)
        eigenvalue_history.append(eigenvalue)
        
        if convergence < tol:
            break
            
        x = x_new
    
    return eigenvalue, x, convergence_history, eigenvalue_history

# Test on a small matrix
A_test = np.array([[4, 1, 1], [1, 3, 0], [1, 0, 2]])
eigval, eigvec, conv_hist, eigval_hist = power_iteration_with_monitoring(A_test)

# True eigenvalue for comparison
true_eigvals = np.linalg.eigvals(A_test)
true_dominant = np.max(np.abs(true_eigvals))

print("Power Iteration Results:")
print(f"Computed eigenvalue: {eigval:.6f}")
print(f"True dominant eigenvalue: {true_dominant:.6f}")
print(f"Error: {abs(eigval - true_dominant):.2e}")
print(f"Iterations: {len(conv_hist)}")

# Plot convergence
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.semilogy(conv_hist)
plt.xlabel('Iteration')
plt.ylabel('Convergence Metric')
plt.title('Convergence History')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(eigval_hist)
plt.axhline(y=true_dominant, color='r', linestyle='--', label='True value')
plt.xlabel('Iteration')
plt.ylabel('Eigenvalue Estimate')
plt.title('Eigenvalue Convergence')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

```{admonition} Exercise 5: Comprehensive Sparse System
:class: dropdown

Create a complete sparse matrix workflow:
1. Build a 200-bus radial + mesh network (main feeder + cross connections)
2. Construct the Y-bus matrix using sparse format
3. Perform LU factorization and analyze fill-in
4. Solve power flow equations with multiple load scenarios
5. Compare direct vs iterative solver performance
6. Implement simple contingency analysis (remove lines)

```

In [ ]:
# Solution to Exercise 5
# Build network
n_buses = 200
branches = []

# Main radial feeder
for i in range(n_buses - 1):
    z = (0.01 + 0.03j) * np.random.uniform(0.8, 1.2)
    branches.append((i, i+1, 1/z))

# Add mesh connections (every 20 buses)
for i in range(0, n_buses-20, 20):
    for j in range(i+5, min(i+20, n_buses), 5):
        if np.random.rand() > 0.5:  # 50% chance
            z = (0.02 + 0.06j) * np.random.uniform(0.8, 1.2)
            branches.append((i, j, 1/z))

print(f"Network Configuration:")
print(f"Buses: {n_buses}")
print(f"Branches: {len(branches)}")

# Build Y-bus
Y_bus = sp.lil_matrix((n_buses, n_buses), dtype=complex)
for i, j, y in branches:
    Y_bus[i, j] -= y
    Y_bus[j, i] -= y
    Y_bus[i, i] += y
    Y_bus[j, j] += y

Y_bus = Y_bus.tocsr()
print(f"Y-bus: {Y_bus.nnz} non-zeros ({Y_bus.nnz/(n_buses**2)*100:.2f}% dense)")

# LU factorization analysis
lu = sp_linalg.splu(Y_bus.tocsc())
fill_in = (lu.L.nnz + lu.U.nnz - Y_bus.nnz) / Y_bus.nnz * 100
print(f"\nLU Factorization:")
print(f"Fill-in: {fill_in:.1f}%")

# Multiple load scenarios
n_scenarios = 10
load_scenarios = np.random.uniform(0.5, 1.5, (n_buses, n_scenarios))
load_scenarios[0, :] = 0  # Slack bus

# Solve with direct method
print(f"\nSolving {n_scenarios} load scenarios:")
start = time.time()
V_direct = np.zeros((n_buses, n_scenarios), dtype=complex)
for i in range(n_scenarios):
    I = load_scenarios[:, i] * np.exp(1j * np.random.uniform(-0.5, 0.5, n_buses))
    V_direct[:, i] = lu.solve(I)
time_direct = time.time() - start

# Solve with iterative method
start = time.time()
V_iterative = np.zeros((n_buses, n_scenarios), dtype=complex)
for i in range(n_scenarios):
    I = load_scenarios[:, i] * np.exp(1j * np.random.uniform(-0.5, 0.5, n_buses))
    V_iterative[:, i], _ = sp_linalg.gmres(Y_bus, I, tol=1e-8)
time_iterative = time.time() - start

print(f"Direct (LU): {time_direct*1000:.1f} ms")
print(f"Iterative (GMRES): {time_iterative*1000:.1f} ms")
print(f"Direct faster by: {time_iterative/time_direct:.1f}x")

# Verify accuracy
max_diff = np.max(np.abs(V_direct - V_iterative))
print(f"\nMax difference: {max_diff:.2e}")

# Contingency analysis
print(f"\nContingency Analysis:")
# Remove a critical line
critical_line = len(branches) // 2
i_remove, j_remove, y_remove = branches[critical_line]

# Create contingency Y-bus
Y_contingency = Y_bus.tolil()
Y_contingency[i_remove, j_remove] += y_remove
Y_contingency[j_remove, i_remove] += y_remove
Y_contingency[i_remove, i_remove] -= y_remove
Y_contingency[j_remove, j_remove] -= y_remove
Y_contingency = Y_contingency.tocsr()

# Check if system is still connected
n_components, labels = csgraph.connected_components(Y_contingency, directed=False)
print(f"Line {i_remove}-{j_remove} removed")
print(f"Connected components: {n_components}")

if n_components == 1:
    # Solve contingency case
    I_test = load_scenarios[:, 0] * np.exp(1j * np.random.uniform(-0.5, 0.5, n_buses))
    V_normal = sp_linalg.spsolve(Y_bus, I_test)
    V_contingency = sp_linalg.spsolve(Y_contingency, I_test)
    
    # Voltage deviation
    V_deviation = np.abs(V_contingency) - np.abs(V_normal)
    worst_bus = np.argmin(V_deviation)
    
    print(f"\nVoltage impact:")
    print(f"Worst affected bus: {worst_bus}")
    print(f"Voltage drop: {V_deviation[worst_bus]:.3f} p.u.")
    print(f"Average voltage drop: {np.mean(V_deviation):.3f} p.u.")
else:
    print("System islanded - contingency creates disconnected network")

# Performance summary
print(f"\nPerformance Summary:")
print(f"Sparse matrix operations enable analysis of {n_buses}-bus system")
print(f"Memory savings: ~{(1 - Y_bus.nnz/(n_buses**2))*100:.0f}%")
print(f"LU factorization enables fast multi-scenario analysis")

## Summary

This advanced NumPy lesson has prepared you for the numerical methods in Module 04 by covering:

**Sparse Matrices**: We learned how to create, manipulate, and efficiently operate on sparse matrices that are ubiquitous in power system analysis. The memory and computational savings are essential for realistic system sizes.

**Advanced Linear Algebra**: LU factorization, iterative solvers, and eigenvalue analysis form the mathematical foundation for power flow, stability analysis, and optimization in Module 04.

**Performance Optimization**: Understanding memory layout, vectorization, and NumPy's optimized functions enables you to write efficient code for large-scale power system problems.

**Numerical Stability**: Concepts like matrix conditioning and iterative refinement are crucial for robust power system software that handles ill-conditioned problems.

These advanced techniques bridge the gap between basic array operations and the sophisticated numerical methods you'll implement in Module 04. The skills learned here will enable you to tackle real-world power system problems efficiently.

```{admonition} Preparation for Module 04
:class: tip

Before starting Module 04, ensure you're comfortable with:
1. Creating and manipulating sparse matrices in different formats
2. Solving large linear systems using both direct and iterative methods
3. Understanding the performance implications of different approaches
4. Building Jacobian matrices and understanding their structure
5. Implementing iterative algorithms with convergence monitoring
```